# Pagerank on subgraphs—efficient Monte-Carlo estimation

In this repo you can find the reference code for my novel Subrank algorithm for efficiently computing the Pagerank distribution over $S$ subgraph of $G$.
For the reasoning behind the algorithm, the definition and the analysis, I invite the interested reader to [read the paper](https://pippellia.com/pippellia/Social+Graph/Pagerank+on+subgraphs%E2%80%94efficient+Monte-Carlo+estimation).

To play with it, follow these steps:

## Step 0: Build and store the Graph

In [71]:
# Imports
from nostr_dvm.utils.wot_utils import build_network_from, save_network, load_network, get_mc_pagerank, get_subrank
from nostr_sdk import  PublicKey
import time
import networkx as nx
import random
from itertools import islice



In [75]:
user = '99bb5591c9116600f845107d31f9b59e2f7c7e09a1ff802e84f1d43da557ca64'
show_results_num = 20

In [14]:
index_map, network_graph = await build_network_from(user, depth=2, max_batch=500, max_time_request=10)
save_network(index_map, network_graph, user)

Step 1: fetching kind 3 events from relays & pre-processing
current network: 44029 npubs
Finished in 39.349228858947754
 > index_map_99bb5591c9116600f845107d31f9b59e2f7c7e09a1ff802e84f1d43da557ca64.json
 > network_graph_99bb5591c9116600f845107d31f9b59e2f7c7e09a1ff802e84f1d43da557ca64.json


## Step 1: load the graph database

First, you have to load the networkx graph database into memory by running the following code.

In [76]:
# loading the database
print('loading the database...')
tic = time.time()

index_map, G = load_network(user)

toc = time.time()
print(f'finished in {toc-tic} seconds')

loading the database...
finished in 0.9305830001831055 seconds


## Step 2: Compute Pagerank over $G$

Compute the pagerank over $G$ by using the networkx built-in pagerank function that uses the power iteration method.
This vector will be considered as the real Pagerank vector and will be used to compute the errors of the Monte-Carlo algorithm.

In [77]:
# computing the pagerank
print('computing global pagerank...')
tic = time.time()

p_G = nx.pagerank(G, tol=1e-12)
for item in islice(p_G, show_results_num): 
    print(next((PublicKey.parse(pubkey).to_bech32() for pubkey, id in index_map.items() if id == item), None) + " " + str(p_G[item]))
    
toc = time.time()
print(f'finished in {toc-tic} seconds')

computing global pagerank...
npub1nxa4tywfz9nqp7z9zp7nr7d4nchhclsf58lcqt5y782rmf2hefjquaa6q8 8.17420152364383e-05
npub1aeh2zw4elewy5682lxc6xnlqzjnxksq303gwu2npfaxd49vmde6qcq4nwx 5.782689313706971e-05
npub10fu0hlkx3s4n4dsgfu0cpqephga4afr4qtzpz9vsyqf7vj88v2yqdp8vp4 4.2360772179521104e-05
npub1zafcms4xya5ap9zr7xxr0jlrtrattwlesytn2s42030lzu0dwlzqpd26k5 4.714319787019475e-05
npub1e30jt8crv6phnrj22gr3mwuhywrs9lak7ry94akjw0ydm0juptas5xmkwq 2.8456329932975766e-05
npub1tr4dstaptd2sp98h7hlysp8qle6mw7wmauhfkgz3rmxdd8ndprusnw2y5g 4.3253896241013506e-05
npub1zuuajd7u3sx8xu92yav9jwxpr839cs0kc3q6t56vd5u9q033xmhsk6c2uc 6.09543251259536e-05
npub1hu3hdctm5nkzd8gslnyedfr5ddz3z547jqcl5j88g4fame2jd08qh6h8nh 7.947486834509654e-05
npub18ams6ewn5aj2n3wt2qawzglx9mr4nzksxhvrdc4gzrecw7n5tvjqctp424 9.098197147408706e-05
npub102a0auqvye3eayugvfwy44un9l477t45uck8s2p08xzpgh784uvslsh7w9 3.646167073617604e-05
npub148xfa6g0e6rqxne88vzfwqag43qk8xuugthptgkzp6qencs6ad9s3rzddg 2.675817761455185e-05
npub1s05p3ha7en49dv8429t

## Step 3: Approximate Pagerank over $G$ using Monte-Carlo

Compute the pagerank over $G$ using a simple Monte-Carlo implementation and compute the L1 error.
This step is essential because it returns the csr-matrix `walk_visited_count`, that will be used later by the Subrank algorithm.

In [78]:


# number of the random walks per node
R = 10

# fix the order of the nodes
nodelist = list(G.nodes())

tic = time.time()

# perform the random walks and get the monte-carlo pagerank
walk_visited_count, mc_pagerank = get_mc_pagerank(G, R, nodelist)

for item in islice(mc_pagerank, show_results_num): 
    print(next((PublicKey.parse(pubkey).to_bech32() for pubkey, id in index_map.items() if id == item), None) + " " + str(mc_pagerank[item]))
    


toc = time.time()
print(f'performed random walks in {toc-tic} seconds')

# computing the L1 error
error_G_mc = sum( abs(p_G[node] - mc_pagerank[node])
                  for node in G.nodes() )

print(f'error pagerank vs mc pagerank in G = {error_G_mc}')

progress = 100%       
Total walks performed:  440290
npub1nxa4tywfz9nqp7z9zp7nr7d4nchhclsf58lcqt5y782rmf2hefjquaa6q8 6.396075897159921e-05
npub1aeh2zw4elewy5682lxc6xnlqzjnxksq303gwu2npfaxd49vmde6qcq4nwx 4.6316411669089085e-05
npub10fu0hlkx3s4n4dsgfu0cpqephga4afr4qtzpz9vsyqf7vj88v2yqdp8vp4 3.969978143064779e-05
npub1zafcms4xya5ap9zr7xxr0jlrtrattwlesytn2s42030lzu0dwlzqpd26k5 4.852195508190285e-05
npub1e30jt8crv6phnrj22gr3mwuhywrs9lak7ry94akjw0ydm0juptas5xmkwq 3.087760777939272e-05
npub1tr4dstaptd2sp98h7hlysp8qle6mw7wmauhfkgz3rmxdd8ndprusnw2y5g 4.6316411669089085e-05
npub1zuuajd7u3sx8xu92yav9jwxpr839cs0kc3q6t56vd5u9q033xmhsk6c2uc 5.954967214597168e-05
npub1hu3hdctm5nkzd8gslnyedfr5ddz3z547jqcl5j88g4fame2jd08qh6h8nh 0.00012130488770475713
npub18ams6ewn5aj2n3wt2qawzglx9mr4nzksxhvrdc4gzrecw7n5tvjqctp424 7.278293262285428e-05
npub102a0auqvye3eayugvfwy44un9l477t45uck8s2p08xzpgh784uvslsh7w9 3.308315119220649e-05
npub148xfa6g0e6rqxne88vzfwqag43qk8xuugthptgkzp6qencs6ad9s3rzddg 2.205543412813766e-

## Step 4: Select random subgraph $S$ and compute its Pagerank distribution

Select a random subgraph $S$ consisting of 50k nodes, and compute its Pagerank distribution.

In [79]:
# selecting random subgraph S
S_nodes = set(random.sample(list(G.nodes()), k=500)) #50000
S = G.subgraph(S_nodes).copy()

# computing pagerank over S
print('computing local pagerank...')
tic = time.time()

p_S = nx.pagerank(S, tol=1e-12)

for item in islice(p_S, show_results_num): 
    print(next((PublicKey.parse(pubkey).to_bech32() for pubkey, id in index_map.items() if id == item), None) + " " + str(p_S[item]))
    

toc = time.time()
print(f'finished in {toc-tic} seconds')

computing local pagerank...
npub1w4wzw7kw55f8pn79294336gunrcghj7p93zprjmujwmd7e2ku6wqlctnak 0.0019403122617013325
npub1z7a4z26ety804299p4p48q5v9mnw0jq68x9tqurzfhn88x95zg3qqrjr5d 0.0019403122617013325
npub1942jsnasalxfefjevx27g093rucxjjran6vt0t90nvhs7ff74p9s76dsgu 0.0019403122617013325
npub1w0j5jrhjlxj024ac7mgv0gpf45k2tjsdtrvcmqk33ksc9cnvsqkqmg6fcn 0.0019403122617013325
npub13eh4qjfnkyc52939wejgrdzur57hcut864wfqe03vquveavrch6sgyds4q 0.0019403122617013325
npub1gywa7ftf537ry82tmvtqn0tn4vqw567hzv2gux7wyryf93699jhqt3qrws 0.0019403122617013325
npub187q40r5ymawxz57dxss36h4urxd78mg3zwnl62q5mdpj57tn9xeq74jrlz 0.0021599470503889952
npub1qlk8uk9dvmzv202uju3t6wpk3skgpreg7szp2zaqt3wslvnkw23sj4ggu5 0.0019403122617013325
npub1a3kwv96mlffyp6xlg08dlqdxgmh9vl9kjce40q0qk0x5a3hv7hxsch7mft 0.0019403122617013325
npub1sjes9vqsdnk0qtfkulfvjxp4wgwn9df5fe9xceyfktx2vakle0tq8lyswv 0.0019403122617013325
npub1qx6kgzfm3z5qlffxnunsgnsaagwcxt6klx6q7gry56arj5r0yl3qgkqf4d 0.0019403122617013325
npub10xllzv9d6ds4vw4pt7x3y

## Step 5: Approximate Pagerank over $S$ using Subrank

Run the Subrank algorithm to approximate the Pagerank over $S$ subgraph of $G$. Then compute the L1 error.

In [80]:
# computing subrank
print('computing subrank over S...')
tic = time.time()

pr = nx.pagerank(G)
for item in islice(pr, show_results_num): 
    print(next((PublicKey.parse(pubkey).to_bech32() for pubkey, id in index_map.items() if id == item), None) + " " + str(pr[item]))
    


print(f'performed random walks in {toc-tic} seconds')

# computing the L1 error
error_S_subrank = sum( abs(p_S[node] - pr[node])
                      for node in S_nodes )

print(f'error pagerank vs subrank in S = {error_S_subrank}')

computing subrank over S...
npub1nxa4tywfz9nqp7z9zp7nr7d4nchhclsf58lcqt5y782rmf2hefjquaa6q8 7.340804412661138e-05
npub1aeh2zw4elewy5682lxc6xnlqzjnxksq303gwu2npfaxd49vmde6qcq4nwx 4.826197846937513e-05
npub10fu0hlkx3s4n4dsgfu0cpqephga4afr4qtzpz9vsyqf7vj88v2yqdp8vp4 3.6176643624020644e-05
npub1zafcms4xya5ap9zr7xxr0jlrtrattwlesytn2s42030lzu0dwlzqpd26k5 3.9090310488700045e-05
npub1e30jt8crv6phnrj22gr3mwuhywrs9lak7ry94akjw0ydm0juptas5xmkwq 2.6394042370260256e-05
npub1tr4dstaptd2sp98h7hlysp8qle6mw7wmauhfkgz3rmxdd8ndprusnw2y5g 3.747815570363107e-05
npub1zuuajd7u3sx8xu92yav9jwxpr839cs0kc3q6t56vd5u9q033xmhsk6c2uc 5.013747945548998e-05
npub1hu3hdctm5nkzd8gslnyedfr5ddz3z547jqcl5j88g4fame2jd08qh6h8nh 6.260807013551759e-05
npub18ams6ewn5aj2n3wt2qawzglx9mr4nzksxhvrdc4gzrecw7n5tvjqctp424 7.07661040710278e-05
npub102a0auqvye3eayugvfwy44un9l477t45uck8s2p08xzpgh784uvslsh7w9 3.270039456677615e-05
npub148xfa6g0e6rqxne88vzfwqag43qk8xuugthptgkzp6qencs6ad9s3rzddg 2.5634274760323094e-05
npub1s05p3ha7en49dv8429

In [81]:
# computing subrank
print('computing subrank over S...')
tic = time.time()

subrank = get_subrank(S, G, walk_visited_count, nodelist)
for item in islice(subrank, show_results_num): 
    print(next((PublicKey.parse(pubkey).to_bech32() for pubkey, id in index_map.items() if id == item), None) + " " + str(subrank[item]))
    


print(f'performed random walks in {toc-tic} seconds')

# computing the L1 error
error_S_subrank = sum( abs(p_S[node] - subrank[node])
                      for node in S_nodes )

print(f'error pagerank vs subrank in S = {error_S_subrank}')

computing subrank over S...
walks performed = 118
npub1w4wzw7kw55f8pn79294336gunrcghj7p93zprjmujwmd7e2ku6wqlctnak 0.0019409937888198758
npub1z7a4z26ety804299p4p48q5v9mnw0jq68x9tqurzfhn88x95zg3qqrjr5d 0.0019409937888198758
npub1942jsnasalxfefjevx27g093rucxjjran6vt0t90nvhs7ff74p9s76dsgu 0.0019409937888198758
npub1w0j5jrhjlxj024ac7mgv0gpf45k2tjsdtrvcmqk33ksc9cnvsqkqmg6fcn 0.0019409937888198758
npub13eh4qjfnkyc52939wejgrdzur57hcut864wfqe03vquveavrch6sgyds4q 0.0019409937888198758
npub1gywa7ftf537ry82tmvtqn0tn4vqw567hzv2gux7wyryf93699jhqt3qrws 0.0019409937888198758
npub187q40r5ymawxz57dxss36h4urxd78mg3zwnl62q5mdpj57tn9xeq74jrlz 0.0021350931677018635
npub1qlk8uk9dvmzv202uju3t6wpk3skgpreg7szp2zaqt3wslvnkw23sj4ggu5 0.0019409937888198758
npub1a3kwv96mlffyp6xlg08dlqdxgmh9vl9kjce40q0qk0x5a3hv7hxsch7mft 0.0019409937888198758
npub1sjes9vqsdnk0qtfkulfvjxp4wgwn9df5fe9xceyfktx2vakle0tq8lyswv 0.0019409937888198758
npub1qx6kgzfm3z5qlffxnunsgnsaagwcxt6klx6q7gry56arj5r0yl3qgkqf4d 0.0019409937888198758
npub

## Step 6: Approximate Pagerank over $S$ using Monte-Carlo naive recomputation

Run the Monte-Carlo Pagerank algorithm on $S$ as a reference for the number of random walks required and the error achieved.

In [82]:
# computing the monte-carlo pagerank 
print('computing naive monte-carlo pagerank over S')
tic = time.time()

_, mc_pagerank_S_naive = get_mc_pagerank(S,R)

for item in islice(mc_pagerank_S_naive, show_results_num): 
    print(next((PublicKey.parse(pubkey).to_bech32() for pubkey, id in index_map.items() if id == item), None) + " " + str(mc_pagerank_S_naive[item]))
    



toc = time.time()
print(f'finished in {toc-tic} seconds')

# computing the L1 error
error_S_naive = sum( abs(p_S[node] - mc_pagerank_S_naive[node])
                      for node in S.nodes())

print(f'error pagerank vs mc pagerank in S = {error_S_naive}')

computing naive monte-carlo pagerank over S
progress = 100%       
Total walks performed:  5000
npub1w4wzw7kw55f8pn79294336gunrcghj7p93zprjmujwmd7e2ku6wqlctnak 0.0019372336303758234
npub1z7a4z26ety804299p4p48q5v9mnw0jq68x9tqurzfhn88x95zg3qqrjr5d 0.0019372336303758234
npub1942jsnasalxfefjevx27g093rucxjjran6vt0t90nvhs7ff74p9s76dsgu 0.0019372336303758234
npub1w0j5jrhjlxj024ac7mgv0gpf45k2tjsdtrvcmqk33ksc9cnvsqkqmg6fcn 0.0019372336303758234
npub13eh4qjfnkyc52939wejgrdzur57hcut864wfqe03vquveavrch6sgyds4q 0.0019372336303758234
npub1gywa7ftf537ry82tmvtqn0tn4vqw567hzv2gux7wyryf93699jhqt3qrws 0.0019372336303758234
npub187q40r5ymawxz57dxss36h4urxd78mg3zwnl62q5mdpj57tn9xeq74jrlz 0.0019372336303758234
npub1qlk8uk9dvmzv202uju3t6wpk3skgpreg7szp2zaqt3wslvnkw23sj4ggu5 0.0019372336303758234
npub1a3kwv96mlffyp6xlg08dlqdxgmh9vl9kjce40q0qk0x5a3hv7hxsch7mft 0.0019372336303758234
npub1sjes9vqsdnk0qtfkulfvjxp4wgwn9df5fe9xceyfktx2vakle0tq8lyswv 0.0019372336303758234
npub1qx6kgzfm3z5qlffxnunsgnsaagwcxt6klx6q7gr